In [1]:
"""
mixedPrees = []
sentenceOnly = []

with open("big-cleanMixedPresFinalThree.txt", "r") as file1, open("big-cleanSentenceOnlyFinal.txt", "r") as file2:

  mixedPrees = file1.readlines()
  sentenceOnly = file2.readlines()

file1.close()
file2.close()

with open("train.txt", "w") as f:

  for i in range(len(mixedPrees)):
    f.write(sentenceOnly[i])
    f.write(mixedPrees[i])

f.close()

"""



'\nmixedPrees = []\nsentenceOnly = []\n\nwith open("big-cleanMixedPresFinalThree.txt", "r") as file1, open("big-cleanSentenceOnlyFinal.txt", "r") as file2:\n\n  mixedPrees = file1.readlines()\n  sentenceOnly = file2.readlines()\n\nfile1.close()\nfile2.close()\n\nwith open("train.txt", "w") as f:\n\n  for i in range(len(mixedPrees)):\n    f.write(sentenceOnly[i])\n    f.write(mixedPrees[i])\n\nf.close()\n\n'

In [2]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import transformers

print(transformers.__version__)

4.30.2


In [4]:
from datasets import load_dataset

datasets = load_dataset("text", data_files={"train": "train.txt", "validation": "testData.txt"})

  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
datasets["train"][10]

{'text': 'Display fireworks aren\'t like the over-the-counter fireworks you can buy in the store, that use chemical fuses and cardboard/paper launchers.  Typically, fireworks are fired from metal mortars.  Black powder or other long-burning explosives are packed in the bottom with an electronic ignitor, wired to an electronic control board.  The firework itself consists of a long chemical fuse, a firecracker type explosive core, binder, a paper container, and pieces of metal called "stars".  When the lifting charge is ignited, it lights the fuse, which times the firecracker core, allowing the firework to get to altitude.  The center core explodes, dispersing the packaging and igniting the stars.  The position and distribution of the stars within the firework define the shape of the explosion - radial, spherical, pouring effects, smily faces, etc.  Fireworks with tails have an segment that is ignited and burns immediately (or shortly after lift-off) and burns brightly as firework rises 

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [6]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [7]:
show_random_elements(datasets["train"])

In [8]:
model_checkpoint = "distilgpt2"

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [10]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [11]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

In [12]:
tokenized_datasets["train"][1]

{'input_ids': [26028,
  326,
  29842,
  561,
  46592,
  290,
  21151,
  866,
  656,
  257,
  2060,
  5858,
  13,
  1318,
  389,
  3294,
  13917,
  1944,
  287,
  262,
  3074,
  13,
  383,
  13917,
  389,
  287,
  6268,
  290,
  15788,
  278,
  1123,
  584,
  13,
  383,
  17661,
  286,
  262,
  13917,
  2482,
  287,
  257,
  2060,
  5858,
  9978,
  13,
  11014,
  597,
  1611,
  286,
  8977,
  6279,
  532,
  3584,
  611,
  262,
  8977,
  318,
  42864,
  1576,
  357,
  72,
  13,
  68,
  1539,
  612,
  338,
  407,
  881,
  286,
  340,
  828,
  262,
  1661,
  38765,
  714,
  307,
  2392,
  621,
  262,
  2479,
  286,
  262,
  5440,
  13,
  1318,
  318,
  257,
  5885,
  286,
  257,
  8977,
  6279,
  4683,
  287,
  262,
  6881,
  13,
  383,
  2479,
  286,
  262,
  5440,
  318,
  2383,
  287,
  13213,
  262,
  1661,
  38765,
  286,
  262,
  8977,
  6279,
  13,
  383,
  12109,
  286,
  262,
  8977,
  6279,
  10975,
  663,
  1661,
  38765,
  13,
  464,
  4931,
  286,
  257,
  8977,
  6279,
  460,

In [13]:
# block_size = tokenizer.model_max_length
block_size = 128

In [14]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [15]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [16]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

"), they needn't be aligned.  But eventually, they'll be aligned.Rings can only effectively extend out to the Roche Radius - beyond that, their self-gravity will result in them accumulating into moons.  This has probably happened with Saturn, at least (which has rings extending to the Roche Radius, then small, icy moons of the same composition beyond. Rings that intercepted would collide and damp down into a single ring. There are multiple rings present in the situation. The rings are in motion and intercepting each other. The collision of the rings results in a single ring formation. Similar any kind of dust cloud - although"

In [17]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [18]:
from transformers import Trainer, TrainingArguments

In [19]:
!pip install accelerate -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-eli5",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [22]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.134600,3.862621
2,3.020800,3.862549
3,2.960200,3.860460


TrainOutput(global_step=7146, training_loss=3.062610586717821, metrics={'train_runtime': 1584.2885, 'train_samples_per_second': 36.077, 'train_steps_per_second': 4.511, 'total_flos': 1866834634604544.0, 'train_loss': 3.062610586717821, 'epoch': 3.0})

In [23]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 47.49


In [24]:
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.5 MB/s eta 0:00:00


In [26]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"


In [27]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.9 MB/s eta 0:00:00


In [29]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from bert_score import score

tokenizer = AutoTokenizer.from_pretrained('distilgpt2')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

test_data = []

with open('testData.txt', 'r', encoding='utf-8') as file:
    test_data = [line.strip() for line in file if line.strip()]

predictions = []
refs = []

# Function to split string at the middle but not cut a word into half
def split_string(s):
    if ' ' not in s:  # If the text doesn't contain a space, return the text as is
        return s, ''
    else:
        midpoint = len(s) // 2  # Integer division to get the middle index
        while midpoint < len(s) - 1 and s[midpoint] != ' ':
            midpoint += 1
        return s[:midpoint], s[midpoint:]

i = 0
# Generate predictions
for line in test_data[:200]:
    prompt, expected_completion = split_string(line)
    inputs = tokenizer.encode(prompt, return_tensors='pt').to(device)
    outputs = model.generate(inputs, max_length=250, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    generated_completion = tokenizer.decode(outputs[0])[len(tokenizer.decode(inputs[0])):]
    print(generated_completion.strip())
    print(expected_completion.strip())
    predictions.append(generated_completion.strip())
    refs.append(expected_completion.strip())
    print(i)
    i = i + 1

print(len(hyps))
print(len(predictions))
print(len(refs))
# Calculate BERTScore
hyps = [pred for pred in predictions if pred]

P, R, F1 = score(predictions, refs, lang='en', model_type='bert-base-uncased')

print(f"Mean F1 BERTScore: {F1.mean().item()}")


our brain. Pain and pleasure are both literally "just in your mind". Pain and pleasure are both literally "just in your mind". Pain and pleasure are both literally "just in your mind". Pain and pleasure are both literally "just in your mind". Pain and pleasure are both literally "just in your mind". Pain and pleasure are both literally "just in your mind". Pain and pleasure are both literally "just in your mind". Pain and pleasure are both literally "just in your mind". Pain and pleasure are both literally "just in your mind". Pain and pleasure are both literally "just in your mind". Pain and pleasure are both literally "just in your mind". Pain and pleasure are both literally "just in your mind". Pain and pleasure are both literally "just in your mind". Pain and pleasure are both literally "just in your mind". Pain and pleasure are both literally "just in your mind". Pain and pleasure are both literally "just in your mind". Pain and pleasure are both literally "just in your mind".
our

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Mean F1 BERTScore: 0.33530914783477783
